#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 data and use them to predict the outcome of cases from the last 30 days

In [13]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=16037 sha256=7e2a12aa461bdf50cf0af98745c4614167067ad448a3d27b884c2dd9c9b1dbb2
  Stored in directory: /tmp/pip-ephem-wheel-cache-_kewmx4l/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [14]:
import os

In [15]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree


## Get latest file URLS and Current Date Ranges

In [16]:
latest_URLS = Boston311LogReg.get311URLs()

In [17]:
print(latest_URLS)

{'2023': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmpn3jozxm6.csv', '2022': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv', '2021': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv', '2020': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv', '2019': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv', '2018': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv', '2017': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/300221

In [18]:
from datetime import datetime, timedelta
now = datetime.now()
thirty_days = timedelta(days=30)
thirty_days_ago = now - thirty_days
today_datestring = now.strftime("%Y-%m-%d")
thirty_days_ago_datestring = thirty_days_ago.strftime("%Y-%m-%d")
tomorrow_datestring = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

print(today_datestring, thirty_days_ago_datestring, tomorrow_datestring)

2023-09-11 2023-08-12 2023-09-12


In [19]:
#set model folder constant
MODEL_FOLDER = './daily_models'

##Define several models

In [20]:
linear_tree_model = Boston311SurvDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type','queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0,
                                      'survivalTimeFill':tomorrow_datestring},
                            files_dict=latest_URLS)

In [21]:
logistic_model = Boston311LogReg(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [22]:
logistic_tree_model = Boston311EventDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [23]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [24]:
logistic_tree_model.model_type

'Boston311EventDecTree'

## Train several models

In [25]:
logistic_tree_model.run_pipeline()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:255: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:255: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1]
Starting Training at 2023-09-11 18:43:15.128570
Testing accuracy: 0.9591422670392419
Ending Training at 2023-09-11 18:43:41.555283
Training took 0:00:26.426713


In [26]:
logistic_model.run_pipeline()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:255: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:255: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1]
Starting Training at 2023-09-11 18:44:37.583257
Epoch 1/10
8231/8231 [==============================] - 6s 656us/step - loss: 0.1857 - accuracy: 0.9340 - val_loss: 0.1221 - val_accuracy: 0.9489
Epoch 2/10
8231/8231 [==============================] - 5s 634us/step - loss: 0.1123 - accuracy: 0.9526 - val_loss: 0.1081 - val_accuracy: 0.9541
Epoch 3/10
8231/8231 [==============================] - 6s 679us/step - loss: 0.1049 - accuracy: 0.9561 - val_loss: 0.1046 - val_accuracy: 0.9564
Epoch 4/10
8231/8231 [==============================] - 5s 650us/step - loss: 0.1026 - accuracy: 0.9572 - val_loss: 0.1031 - val_accuracy: 0.9569
Epoch 5/10
8231/8231 [==============================] - 5s 637us/step - loss: 0.1015 - accuracy: 0.9576 - val_loss: 0.1025 - val_accuracy: 0.9573
Epoch 6/10
823

In [27]:
import gc
gc.collect()

2351

In [28]:
linear_tree_model.run_pipeline()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:255: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:255: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1]
Starting Training at 2023-09-11 18:46:29.368524
Testing accuracy: 0.6966589721783502
Ending Training at 2023-09-11 18:46:37.829860
Training took 0:00:08.461336


In [30]:
import datetime

def save_model_to_dir(model, folder_name):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)

# List of models
models = [linear_tree_model, logistic_tree_model, logistic_model]


# Iterate over models and save
for model in models:
    save_model_to_dir(model, model.model_type)
